A/B testing, also known as split testing, is a statistical method used in data science to compare two versions of a product, webpage, or marketing campaign to determine which one performs better based on a specific metric. This approach allows data scientists and marketers to make data-driven decisions rather than relying on intuition or guesswork.

## A/B Testing Procedure

1. __Problem Statement__ - What is the goal of the experiment ?
2. __Hypothesis Testing__ - What result do you hypothesize from the experiment?
3. __Design the Experiment__ - What are the experiment parameters ?
4. __Run the Experiment__ - What are the requirements for running it ?
5. __Validity Checks__ - Did the experiment run soundly without errors or bias?
6. __Interpret the Results__ - Is the metric significant statistically and practically?
7. __Launch Decision__ - Take action based on the experiment results

## Tips

- Talk about the business goal first (user journey)
- Use the user funnel to create the success metric
- A success metric must be: measurable, attributable, sensitive and timely

## Example

A web store wants to change the product ranking recommendation system.

- __Success Metric__: revenue per day per user
- __Null Hypothesis ($Ho$)__: the average revenue per day per user between the baseline and variant ranking algorithms are the same